In [1]:
!pip install hume

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.9/355.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 13.1 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [6]:
from hume import HumeClient

# Initialize the client with your API key
client = HumeClient(api_key="")
client.empathic_voice.configs.list_configs()
# # Example: Using the empathic voice interface
# response = client.empathic_voice.send_text(
#     text="Hello, how are you?",
#     voice="default"
# )
# print(response)

SyncPager(get_next=<function RawConfigsClient.list_configs.<locals>.<lambda> at 0x7864f1102520>, has_next=True, items=[], response=<hume.core.http_response.BaseHttpResponse object at 0x7864f159cf20>)

In [9]:
from hume import AsyncHumeClient

hume = AsyncHumeClient(api_key="")

import base64
from hume.empathic_voice.chat.audio.audio_utilities import play_audio_streaming
from hume.tts import PostedUtterance, PostedUtteranceVoiceWithName

utterance = PostedUtterance(
    text="Dogs became domesticated between 23,000 and 30,000 years ago.",
    voice=PostedUtteranceVoiceWithName(name='Ava Song', provider='HUME_AI')
)

stream = hume.tts.synthesize_json_streaming(
    utterances=[utterance],
    strip_headers=True,
    version="1"
)

await play_audio_streaming(base64.b64decode(chunk.audio) async for chunk in stream)

RuntimeError: pip install "hume[microphone]" ‑‑ or drop audio playback entirely

In [8]:
!pip install "hume[microphone]"

In [12]:
# Install dependencies
# !pip install requests

import os
import requests
import time

BASE_URL = "https://tavusapi.com/v2"
headers = {
    "Content-Type": "application/json",
    "x-api-key": ""
}

# Replace with your IDs from Tavus dashboard
REPLICA_ID = ""
PERSONA_ID = ""

# 1. (Optional) Create or ensure persona has TTS layer configured
#    If you already have a persona, you can skip this.
payload_persona = {
    "persona_name": "TTS Persona",
    "default_replica_id": REPLICA_ID,
    "layers": {
        "tts": {
            "tts_engine": "cartesia",
            "external_voice_id": None,
            "voice_settings": {
                "speed": 1.0,
                "emotion": ["neutral"]
            },
            "tts_model_name": "sonic",
            "tts_emotion_control": False
        }
    }
}
resp = requests.post(f"{BASE_URL}/personas", json=payload_persona, headers=headers)
print("Create persona status:", resp.status_code, resp.text)
# Set PERSONA_ID from resp if newly created

# 2. Use conversation endpoint to send text and get back audio
text = "Hello world, this is a test of Tavus text to speech."
payload_conv = {
    "replica_id": REPLICA_ID,
    "persona_id": PERSONA_ID,
    "properties": {
        "language": "English"
    },
    "messages": [
        {
            "participant": "user",
            "content": text
        }
    ]
}
resp2 = requests.post(f"{BASE_URL}/conversations", json=payload_conv, headers=headers)
print("Create conversation status:", resp2.status_code, resp2.text)
if resp2.status_code != 200:
    raise RuntimeError("Conversation creation failed")

data = resp2.json().get("data", [{}])[0]
# The response may contain an audio URL or similar field
audio_url = data.get("audio_url") or data.get("url")
print("Audio URL:", audio_url)

if audio_url:
    r_audio = requests.get(audio_url)
    with open("output_audio.mp3", "wb") as f:
        f.write(r_audio.content)
    print("Saved output_audio.mp3")


Create persona status: 401 {"message":"Invalid access token"}

Create conversation status: 401 {"message":"Invalid access token"}



RuntimeError: Conversation creation failed